# Import des modules

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
import json

C:\Users\joaka\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


# Lecture des données

In [2]:
db = pd.read_csv("../Ars_data_v_2/output_data.csv", encoding="utf-8", low_memory=False)
db.drop(db.columns[0], axis=1, inplace=True)

# Features engineering

In [3]:
code_dep = set(db['code_dep'])
db['PGP19_>60'].fillna(0, inplace=True)
for code in code_dep:
    db_temp = db[db['code_dep'] == code]
    sum_pop = db_temp['PM16'].sum()
    db.loc[(db['code_dep'] == code),'PPMsD16']= db['PM16']/sum_pop
db=db.where(pd.notnull(db), None)
db=db.dropna()

# Séparation des données

In [4]:
var_desc = db[["Nom_com","code_dep","code_commune_INSEE",
               "nom_departement","nom_region","code_postal","code_region","PM16"]] 
var_num = db[["RFM16","TN16","PPMsD16","NSP19","DP16","PGP19","PGP19_>60",
              "PC16_24_64","PC16_15_24","PPop16_<18","PPop16_>65","PRP16"]]
X = var_num

# Calcule des valeurs à afficher

In [5]:
# median_sup = var_num.median()
# std = var_num.std()
# first_q = var_num.quantile(0.25)
# third_q = var_num.quantile(0.75)

In [6]:
# def calcule_indicateur(df):
#     i = 0
#     colnames = df.columns
#     for colname in colnames:
#         df[colname] = df[colname].apply(lambda x: 1 if(x>median[i] + std[i]) else 
#                                         (-1 if(x<median[i] - std[i]) else 0))
#         i+=1
#     return df

In [7]:
# display_value = calcule_indicateur(X)

In [8]:
# col_sum = display_value.sum(axis=1)

In [9]:
# final = pd.concat([var_desc,display_value],axis=1)
# final['Score'] = col_sum/len(var_num.columns)

Pour Théo (Auto Complétion)

In [10]:
# final['Nom_com'].to_csv("liste_des_communes", sep=",")

# Export en json

In [11]:
# d = db.set_index("Nom_com").T.to_dict()

In [12]:
# with open('data.json', 'w', encoding='utf8') as file:
#      file.write(json.dumps(d, ensure_ascii=False))

# Classification

In [13]:
# kmeans = KMeans(n_clusters=12,init='k-means++', n_init=10, random_state=0)

In [14]:
# kmeans.fit(X)

In [15]:
# wcss = []
# for i in range(1, 50):
#     kmeans = KMeans(n_clusters=i,init='k-means++', n_init=10, random_state=0)
#     kmeans.fit(X)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 30), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# Plus proche voisins

In [16]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='kd_tree').fit(X)
distances, indices = nbrs.kneighbors(X)
indices

array([[    0, 20752, 19981, 17452, 14640,   336],
       [    1,  5778, 17396,  2491, 28474, 28902],
       [    2,  4786, 17513, 21510, 16863, 21083],
       ...,
       [29356, 17915, 22615, 18687, 20182, 22654],
       [29357,   151, 25707,  5538, 13833, 23545],
       [29358, 17184,  9424,  6930, 25597,  7136]], dtype=int64)

In [17]:
proche_voisin = []
for i in range(len(db.index)):
    proche_voisin.append(list(db.iloc[indices[i][1:]]['Nom_com']))

In [18]:
db['Proche_voisin'] = proche_voisin

In [19]:
d = db.set_index("Nom_com").T.to_dict()

C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


Extraction des seuils pour clément

In [20]:
# d = {}
# for name in X.columns:
#     median = X[name].median()
#     sd = X[name].std()
#     d[name] = [{'25%':X[name].quantile(0.25),'50%':X[name].quantile(0.5),'75%':X[name].quantile(0.75),'sd':sd}]

In [21]:
# with open('exemple_seuil.json', 'w', encoding='utf8') as file:
#      file.write(json.dumps(d, ensure_ascii=False))

In [22]:
with open('final_data.json', 'w', encoding='utf8') as file:
    file.write(json.dumps(d, ensure_ascii=False))